In [ ]:
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf   #version 1.13.2
import numpy as np
!pip install tflearn
import tflearn            #version 0.3.2
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
with open("intents.json") as json_data: 
    intents = json.load(json_data)   

# import json

# with open(r'/kaggle/input/mental-health-conversational-data/intents.json') as json_data:
#     intents = json.load(json_data)

In [ ]:
words=[]
documents = []
classes = []


ignore = ["?"]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        w = nltk.word_tokenize(pattern) 
        #print(w)
        
        words.extend(w) 
        #print(words)

        documents.append((w, intent["tag"]))
        #print(documents)
        
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])
            #print(classes)

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words))) 

classes = sorted(list(set(classes)))

print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

232 Documents 

80 Classes 

279 Stemmed Words 


In [ ]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []

    pattern_words = doc[0]
    #print(pattern_words)
    
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    #print(pattern_words)

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [ ]:
random.shuffle(training) #Suffling the data or features
training = np.array(training) #Converting training data into numpy array

train_x = list(training[:,0])

train_y = list(training[:,1])

<ipython-input-38-d01ea5ee0cc1>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training) #Converting training data into numpy array


In [ ]:
from tensorflow.python.framework import ops

ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") #Saving the model

Training Step: 28761  | total loss: 0.00096 | time: 0.103s
| Adam | epoch: 992 | loss: 0.00096 - acc: 0.9999 -- iter: 176/232


In [ ]:
import pickle

pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [ ]:
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open("intents.json") as json_data:
    intents = json.load(json_data)

In [ ]:
# Loading the saved model
model.load("./model.tflearn")

In [ ]:
def clean_up_sentence(sentence):

    sentence_words = nltk.word_tokenize(sentence)
    
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

def bow(sentence, words, show_details=False):
    
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [ ]:
#Adding some context to the conversation for better results.

context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):

    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def query(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    #If we have a classification then find the matching intent tag
    if results:
        while results:
            for i in intents['intents']:
                
                if i['tag'] == results[0][0]:
                    
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
query("Hi")

In [ ]:
query("My name is Tanishk")

In [ ]:
query("Who are you?")

In [ ]:
query("Who are you?")

In [ ]:
query("My mom died")

In [ ]:
query("What should I talk about?")

In [ ]:
query("I feel anxious right now")

In [ ]:
query("I am scared!!!")

In [ ]:
query("I love to kill people")

I'm very sorry to hear that but you have so much to look forward to. Please seek help by contacting: 9152987821.
